### Data importation 

### YahooFinance 

In [6]:
pip install pandas numpy yfinance 

Note: you may need to restart the kernel to use updated packages.


## Data Loading

In [7]:
import pandas as pd
import numpy as np
import yfinance as yf

# --- Define Parameters ---
GREEN_TICKERS = ['ENPH', 'TSLA', 'NEE'] # Example Green stocks: Enphase, Tesla, NextEra Energy
BROWN_TICKERS = ['XOM', 'CVX', 'EOG'] # Example Brown stocks: Exxon Mobil, Chevron, EOG Resources

START_DATE = '2015-01-01'
END_DATE = '2025-10-31'
RISK_FREE_RATE_ANNUAL = 0.04 # Example (use T-Bill rate for actual project)

# --- Download Stock Data ---
all_tickers = GREEN_TICKERS + BROWN_TICKERS
raw_price_data = yf.download(all_tickers, start=START_DATE, end=END_DATE)

# Extract adjusted close robustly (handle single- and multi-index columns)
if isinstance(raw_price_data.columns, pd.MultiIndex):
	# Prefer 'Adj Close' or 'Close' on the first level (commonly the 'Price' level)
	if 'Adj Close' in raw_price_data.columns.get_level_values(0):
		price_data = raw_price_data.xs('Adj Close', axis=1, level=0)
	elif 'Close' in raw_price_data.columns.get_level_values(0):
		price_data = raw_price_data.xs('Close', axis=1, level=0)
	else:
		# As a fallback, try the other level (some yfinance variants use reversed ordering)
		if 'Adj Close' in raw_price_data.columns.get_level_values(1):
			price_data = raw_price_data.xs('Adj Close', axis=1, level=1)
		elif 'Close' in raw_price_data.columns.get_level_values(1):
			price_data = raw_price_data.xs('Close', axis=1, level=1)
		else:
			raise KeyError(f"'Adj Close' (or 'Close') not found in downloaded data. Columns: {raw_price_data.columns}")
else:
	# Single-level columns (e.g., when downloading one ticker or if yfinance returns flat frame)
	if 'Adj Close' in raw_price_data.columns:
		price_data = raw_price_data['Adj Close']
	elif 'Close' in raw_price_data.columns:
		price_data = raw_price_data['Close']
	else:
		# If raw_price_data is already a single series (one ticker), use it directly
		if isinstance(raw_price_data, pd.Series):
			price_data = raw_price_data
		else:
			# If columns are already ticker symbols (multiple tickers but no 'Close' level),
			# assume the frame itself represents closing prices.
			cols_as_str = list(map(str, raw_price_data.columns))
			if set(all_tickers).issubset(set(cols_as_str)):
				price_data = raw_price_data
			else:
				raise KeyError(f"'Adj Close' (or 'Close') not found and columns are not tickers. Columns: {raw_price_data.columns}")

# --- Calculate Daily Returns ---
returns_data = price_data.pct_change().dropna()


/var/folders/00/c54qzf757tzdyqs5hr4n94vc0000gn/T/ipykernel_36064/347246425.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_price_data = yf.download(all_tickers, start=START_DATE, end=END_DATE)
[*********************100%***********************]  6 of 6 completed


# Sanity Check 

In [11]:
print(price_data.head())

Ticker            CVX   ENPH        EOG        NEE       TSLA        XOM
Date                                                                    
2015-01-02  70.209541  14.21  67.469185  20.032854  14.620667  57.916904
2015-01-05  67.403168  13.88  63.402317  19.820679  14.006000  56.332176
2015-01-06  67.371994  12.94  61.595627  19.897663  14.085333  56.032719
2015-01-07  67.315857  11.40  61.207985  20.079796  14.063333  56.600468
2015-01-08  68.856262  11.73  63.548618  20.201841  14.041333  57.542576


In [9]:
print(returns_data.head())

Ticker           CVX      ENPH       EOG       NEE      TSLA       XOM
Date                                                                  
2015-01-05 -0.039971 -0.023223 -0.060277 -0.010591 -0.042041 -0.027362
2015-01-06 -0.000462 -0.067723 -0.028496  0.003884  0.005664 -0.005316
2015-01-07 -0.000833 -0.119011 -0.006293  0.009153 -0.001562  0.010132
2015-01-08  0.022883  0.028947  0.038241  0.006078 -0.001564  0.016645
2015-01-09 -0.019926 -0.017050  0.006676 -0.008830 -0.018802 -0.001410


# yFinance CSV daily_returns

In [10]:
portfolio_returns = returns_data.copy()
# save to CSV — provide filename as a string
portfolio_returns.to_csv('daily_returns.csv')
print("Daily returns saved to 'daily_returns.csv'")


Daily returns saved to 'daily_returns.csv'


In [12]:
import os
print(os.getcwd())
print(os.listdir())


/Users/reynoldtakurachoruma/Desktop/Green well vs Brown  
['aligned_data.csv', 'final_notebook.ipynb', '.DS_Store', 'cleaned_market_risk_premium.csv', 'final notebook .ipynb', 'FRED.ipynb', 'rf_daily.csv', 'Market Risk Premium.ipynb', 'basket_metrics.csv', 'Yahoo Finance.ipynb', 'GHG:Carbon Emissions.ipynb', 'daily_returns.csv', 'Data Alignment and Merging.ipynb', 'ghg_macro.csv']
